In [1]:
# Import dependencies
from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': 'C:/Anaconda/webscrapes/chromedriver'}
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('log-level=3')
browser = Browser('chrome', **executable_path, wait_time=3,headless=True,options=chrome_options)

In [5]:
# scrape links to each dealer and place in a list
url = 'https://www.lennox.com/locate/dealer-list/'
browser.visit(url)
html = browser.html
lennox_soup = BeautifulSoup(html, 'html.parser')

dealer_links = []

for i in lennox_soup.find_all(href=re.compile('/locate/dealer')):
    dealer_links.append('https://www.lennox.com'+i.get('href'))
dealer_links = dealer_links[1:]

In [9]:
dealer_links[0:5]

len(dealer_links)


2574

In [7]:
pd.DataFrame(dealer_links).to_csv('data/Lennox_Dealer_List.csv',index=False)

In [8]:
lennox_links = pd.read_csv('data/Lennox_Dealer_List.csv')
lennox_links.head()

,0
0,https://www.lennox.com/locate/dealer/al/semmes...
1,https://www.lennox.com/locate/dealer/al/millbr...
2,https://www.lennox.com/locate/dealer/al/huntsv...
3,https://www.lennox.com/locate/dealer/al/birmin...
4,https://www.lennox.com/locate/dealer/al/huntsv...


In [15]:
import time
start_time = time.time()

df = pd.DataFrame()

for dealer in lennox_links:
    try:
        browser.visit(dealer)
        html = browser.html
        dealer_soup = BeautifulSoup(html, 'html.parser')
        
        name = dealer_soup.find(itemprop='name').get_text(strip=True)
        street = dealer_soup.find(itemprop='streetAddress').get_text(strip=True)
        city = dealer_soup.find(itemprop='addressLocality').get_text(strip=True)
        state = dealer_soup.find(itemprop='addressRegion').get_text(strip=True)
        zipcode = dealer_soup.find(itemprop='PostalCode').get_text(strip=True)
        country = dealer_soup.find(itemprop="addressCountry")['content']
        phone = dealer_soup.find(itemprop='telephone').get_text('[0-9]',strip=True)[-12:]
        lat = dealer_soup.find(itemprop="latitude")['content']
        long = dealer_soup.find(itemprop="longitude")['content']
        try:
            website = dealer_soup.find(attrs={'class':'website ng-binding ng-scope'}).get('href')
        except:
            website = ''
        
    except:
        pass
    dealer_df = {'DealerName':name,
                 'PostalAddress':street,
                 'City':city,
                 'State':state,
                 'ZipCode':zipcode,
                 'Country':country,
                 'Phone':phone,
                 'LocatorLatitude':lat,
                 'LocatorLongitude':long,
                 'Website':website,
                 'Brand':'Lennox'}

    df = df.append(dealer_df,ignore_index=True)
# df
browser.quit()    

df2 = pd.DataFrame([df.DealerName,df.PostalAddress,df.City,df.State,df.ZipCode,df.Country,df.Phone,df.LocatorLatitude,df.LocatorLongitude,df.Website,df.Brand]).T

print("--- %s seconds ---" % (time.time() - start_time))

--- 5314.282999992371 seconds ---


In [20]:
from sqlalchemy import create_engine
import urllib


params = urllib.parse.quote_plus('DRIVER={SQL Server Native Client 10.0};Server=gmlsql2hou; DATABASE=invplannersapp;Trusted_Connection=yes;')

engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
df2.to_sql('GBU_Webscrape',engine,if_exists='append',index=False)

In [12]:
# dealer_links[1:10]

['https://www.lennox.com/locate/dealer/al/semmes/action-htg-ac-and-refrig-inc',
 'https://www.lennox.com/locate/dealer/al/millbrook/air-now-heating-and-cooling-inc',
 'https://www.lennox.com/locate/dealer/al/huntsville/aire-serv-of-huntsville',
 'https://www.lennox.com/locate/dealer/al/birmingham/aireserv-over-the-mountain',
 'https://www.lennox.com/locate/dealer/al/huntsville/all-weather-htg-and-air-cond-inc',
 'https://www.lennox.com/locate/dealer/al/huntsville/annette-hales-indoor-comfort-syste',
 'https://www.lennox.com/locate/dealer/al/heflin/bain-htg-and-a-c-inc',
 'https://www.lennox.com/locate/dealer/al/fairhope/bay-aire-inc',
 'https://www.lennox.com/locate/dealer/al/dothan/bob-woodall-air-care-systems']

In [15]:
# import lennox

# lennox.lennox_scrape(urls=dealer_links[1:10])


Lennox Scrape completed
